In [0]:
# 2.3 DEMO: Implementing Delta Sharing (Databricks-to-Open)

## Provider Workspace - External Client Sharing

**Learning Objectives:**
- Share data with external (non-Databricks) clients
- Create open sharing recipients
- Generate credential files for external access
- Understand the difference between D2D and D2O sharing
- Configure shares for external client consumption

**Scenario:**
You are a data provider company that wants to share analytics data with external partners who may not use Databricks. They will access the data using open-source tools like Python, Power BI, or other Delta Sharing compatible clients.

**Key Difference from D2D:**
- D2O sharing uses credential files (bearer tokens) instead of activation links
- External clients use the delta-sharing Python library or other compatible clients
- No Databricks workspace required for recipients
- OIDC authentication is also available (out of scope for this demo)

In [ ]:
%run ../../setup/00-provider-setup

## Step 1: Create or Reuse Catalog and Tables

We'll reuse the catalog and tables from the previous demo, or create them if they don't exist.

In [ ]:
-- Create catalog and schema (if not exists)
CREATE CATALOG IF NOT EXISTS ${c.catalog_name};
USE CATALOG ${c.catalog_name};

CREATE SCHEMA IF NOT EXISTS ${c.schema_name};

-- Create a table with inventory data for external sharing
CREATE OR REPLACE TABLE ${c.catalog_name}.${c.schema_name}.product_inventory (
  product_id INT,
  product_name STRING,
  category STRING,
  supplier STRING,
  stock_quantity INT,
  unit_cost DECIMAL(10,2),
  last_updated TIMESTAMP
)
COMMENT 'Product inventory for external client access';

-- Insert sample data
INSERT INTO ${c.catalog_name}.${c.schema_name}.product_inventory VALUES
  (201, 'Laptop Pro 15', 'Electronics', 'Tech Distributors Inc', 45, 899.99, current_timestamp()),
  (202, 'Wireless Keyboard', 'Electronics', 'Tech Distributors Inc', 120, 49.99, current_timestamp()),
  (203, 'Office Chair Deluxe', 'Furniture', 'Office Pro Ltd', 30, 249.99, current_timestamp()),
  (204, 'Standing Desk', 'Furniture', 'Office Pro Ltd', 15, 599.99, current_timestamp()),
  (205, 'USB-C Hub', 'Electronics', 'Tech Distributors Inc', 200, 34.99, current_timestamp()),
  (206, 'Monitor 27 inch', 'Electronics', 'Tech Distributors Inc', 60, 299.99, current_timestamp()),
  (207, 'Desk Organizer', 'Office Supplies', 'Supply Chain Co', 150, 19.99, current_timestamp()),
  (208, 'Webcam HD', 'Electronics', 'Tech Distributors Inc', 80, 79.99, current_timestamp());

SELECT * FROM ${c.catalog_name}.${c.schema_name}.product_inventory;

## Step 2: Create a Share for External Clients

Create a new share specifically for external (open) client access.

In [ ]:
-- Create a share for external clients
CREATE SHARE IF NOT EXISTS ${c.share_name}
COMMENT 'Share for external client access to product inventory';

-- Add the inventory table to the share
ALTER SHARE ${c.share_name}
ADD TABLE ${c.catalog_name}.${c.schema_name}.product_inventory
COMMENT 'Product inventory data for external analytics';

-- View the share contents
SHOW ALL IN SHARE ${c.share_name};

## Step 3: Create an Open (External) Recipient

For external clients, we create a recipient without authenticating to a specific Databricks workspace. This generates a credential file (bearer token) that external clients can use.

In [ ]:
-- Create an open recipient (no specific Databricks workspace)
-- Note: For D2O, we don't specify USING ID (which would be for D2D)
CREATE RECIPIENT IF NOT EXISTS ${c.d2o_recipient}
COMMENT 'External client recipient for open sharing';

-- View all recipients
SHOW RECIPIENTS;

## Step 4: Grant Share to Recipient

Grant the share to the external recipient.

In [ ]:
-- Grant the share to the recipient
GRANT SELECT ON SHARE ${c.share_name} TO RECIPIENT ${c.d2o_recipient};

-- Describe the recipient to get the credential download URL
DESCRIBE RECIPIENT ${c.d2o_recipient};

## Step 5: Download and Share the Credential File

**Important:** From the output above, you'll see an `activation_link` URL. 

### For External Clients:
1. **Copy the activation_link URL** from the output above
2. **Open the URL in a browser** - this will download a `.share` credential file
3. **Share this file securely** with your external client
4. The file contains:
   - Share endpoint URL
   - Bearer token for authentication
   - Share name

### Security Notes:
- 🔐 Treat the `.share` file like a password - it provides access to your data
- ⏰ Tokens can be rotated or revoked at any time
- 📧 Share the file through secure channels (encrypted email, secure file transfer)
- 🔄 Consider setting token expiration policies

### Alternative: OIDC Authentication
- **OIDC (OpenID Connect)** provides more secure, enterprise-grade authentication
- Eliminates the need for bearer tokens
- Out of scope for this demo, but recommended for production use
- Requires additional identity provider configuration

## Step 6: Monitor and Manage the Share

You can monitor who is accessing your shared data and manage the recipient.

In [ ]:
-- View all recipients
SHOW RECIPIENTS;

-- View details of specific recipient
SHOW GRANTS ON SHARE ${c.share_name};

-- View share details
DESCRIBE SHARE ${c.share_name};

-- If needed, revoke access
-- REVOKE SELECT ON SHARE ${c.share_name} FROM RECIPIENT ${c.d2o_recipient};

-- If needed, drop the recipient
-- DROP RECIPIENT ${c.d2o_recipient};

## Summary: Databricks-to-Open (D2O) Sharing

### Key Differences from D2D:

| Aspect | D2D (Databricks-to-Databricks) | D2O (Databricks-to-Open) |
|--------|--------------------------------|--------------------------|
| **Recipient Type** | Databricks workspace required | Any client with delta-sharing library |
| **Authentication** | Activation link creates catalog | Credential file (.share) with bearer token |
| **Access Method** | SQL queries in Databricks | Python, Pandas, PowerBI, Tableau, etc. |
| **Recipient Creation** | `CREATE RECIPIENT` with workspace ID | `CREATE RECIPIENT` without workspace ID |
| **Use Case** | Internal Databricks users | External partners, BI tools, data scientists |

### What We Accomplished:
✅ Created an open recipient for external access  
✅ Generated a credential file download URL  
✅ Configured bearer token authentication  
✅ Prepared data for consumption by any delta-sharing client  

### Next Steps:
The recipient can now use the credential file with:
- Python (delta-sharing library)
- Power BI Desktop (via connector)
- Tableau (via connector)
- Any tool supporting the Delta Sharing protocol